# ccdproc-04:   BIAS correction

## Overview
1. Manually substract BIAS 
2. Analysing the results
3. Using master BIAS to correct from BIAS all the images

<pre>
Máster en Astrofísica UCM  -- Técnicas Experimentales en Astrofísica  
Jaime Zamorano and Nicolás Cardiel

This notebook has reproduced parts of the the astropy ccdproc docs: 
https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html#subtract-bias-and-dark

Version 1.0 2021/01/15  
</pre>

Note that the ``astropy`` package should be installed. In this sense, have a look to the
astropy installation description: https://docs.astropy.org/en/stable/install.html.

In [ ]:
from pylab import *
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.io import fits
from astropy import units as u
from astropy.nddata import CCDData
import ccdproc

In [ ]:
# Some parameters for nicer graphs
matplotlib.rcParams['text.usetex']=False
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Verdana']
rcParams['font.size'] = 12.
rcParams['lines.linewidth'] = 4.
rcParams['axes.labelsize'] = 'large'
rcParams['grid.linewidth'] = 1.0
rcParams['grid.linestyle'] = ':'

### Working with images in a directory
We will use the files of the first night of observations at NOT (Nordic Optical Telescope) 2008 that can be downloaded from   ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/N1/
or 
http://guaix.fis.ucm.es/~jaz/master_TEA/observaciones_NOT_2008/N1/

``directory`` should point to the working directory.  


At this step we have already created a combination of BIAS frames with the result of a master DARK that we called zero_N1.fits. This file should be in our working directory that also contents the trimmed images.  

In [ ]:
directory='/Users/jzamorano/Desktop/NOT_2008/N1/'   # change the path to your working directory

Let\'s create a list containing all the FITS files in that directory.  
We asume that the images were already trimmed and their names begin with 't_' (see previous notebooks 'ccdproc_02' and 'ccdproc_03')

In [ ]:
import os
from glob import glob
# os.path.join is a platform-independent way to join two directories
globpath = os.path.join(directory, 't_*.fits')
print(globpath)
# glob searches through directories similar to the Unix shell
filelist = sorted(glob(globpath))
print(filelist[100:105])    # printing only from 10 to 20

### Statistics and display

#### Reading the master  BIAS file 

In [ ]:
filename = 'zero_N1.fits'
master_bias = CCDData.read(directory+filename)

#### Reading some example the files

In [ ]:
science = CCDData.read(filelist[102])    # file with a science observation
onebias = CCDData.read(filelist[9])      # file with a single BIAS frame

#### Analysing the images data

In [ ]:
print('Filename          Object            exp  Mean std min  max')
print(science.header['FILENAME'], science.header['OBJECT'], science.header['EXPTIME'], 
          int(np.mean(science)), int(np.std(science)), np.min(science), np.max(science))
print(onebias.header['FILENAME'], onebias.header['OBJECT'], onebias.header['EXPTIME'], 
          int(np.mean(onebias)), int(np.std(onebias)), np.min(onebias), np.max(onebias))

The science image of this example is one of the three 300s exposure observations of NGC4496A. The maximum value is 65535 which means that there is saturation in some pixels of the image. This is the highest number that can be represented by an unsigned 16-bit binary number (2^16 -1)

#### Display of the example files

In [ ]:
# auxiliary function to display a rectangle and compute mean value within it
def draw_rectangle(ax, image_data, x1, x2, y1, y2, color, text=False):
    ax.plot((x1, x1), (y1, y2), color, lw=1)
    ax.plot((x2, x2), (y1, y2), color, lw=1)
    ax.plot((x1, x2), (y1, y1), color, lw=1)
    ax.plot((x1, x2), (y2, y2), color, lw=1)
    if text:
        media = image_data[y1:y2,x1:x2].mean()
        std   = image_data[y1:y2,x1:x2].std()
        ax.text((x1+x2)/2, y1+(y2-y1)/8, str(int(media)), 
                ha='center', va='center', color=color, fontsize=12)        
        ax.text((x1+x2)/2, y2-(y2-y1)/8, str(round(std,1)), 
                ha='center', va='top', color=color, fontsize=12)
    return media, std

In [ ]:
fig, axarr = plt.subplots(ncols=2, nrows=1, figsize=(12, 9))
ax0 = axarr[0]
img = ax0.imshow(onebias, cmap='gray', origin='low',vmin=340,vmax=380)
ax0.set_xticks([])
ax0.set_yticks([])
ax0.text(600, 100, 'BIAS   '+onebias.header['FILENAME'], fontsize=10, color='w')
divider = make_axes_locatable(ax0)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax) #, label='Number of counts')
draw_rectangle(ax0, onebias.data , 200, 500, 200, 500, color='w',text=True)
ax1 = axarr[1]
img = ax1.imshow(science, cmap='gray', origin='low',vmin=5000,vmax=8000)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.text(600, 100, science.header['FILENAME'], fontsize=10, color='w')
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax) #, label='Number of counts')
draw_rectangle(ax1, science.data , 200, 500, 200, 500, color='w',text=True)


### Manually substracting the master BIAS 

Although we will use ``ccdproc.subtract_bias`` let's substract the master BIAS from both images just to show that the procedure is a simple substraction.

In [ ]:
onebias_minus_bias = np.subtract(onebias,master_bias)
science_minus_bias = np.subtract(science,master_bias)

In [ ]:
fig, axarr = plt.subplots(ncols=2, nrows=1, figsize=(12, 9))
images = [onebias_minus_bias,science_minus_bias]
cuts = [0,10,5000,8000]
for i in range(2):
    ax = axarr[i]
    vmin = cuts[0+2*i]
    vmax = cuts[1+2*i]
    img = ax.imshow(images[i].data, cmap='gray', origin='low',vmin=vmin,vmax=vmax)
    ax.set_xticks([])
    ax.set_yticks([])
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    ax.text(200, 1800, 'using numpy array substract', fontsize=12, color='y')
    fig.colorbar(img, cax=cax) #, label='Number of counts')
    draw_rectangle(ax, images[i] , 200, 500, 200, 500, color='w',text=True)

As expected, the BIAS substracted BIAS has signal around 0 and the standard deviation does not change. For the science data, the result is a frame with lower signal (around 358 counts).

### Using ccdproc_substract_bias 

In [ ]:
bias_subtracted_onebias = ccdproc.subtract_bias(onebias, master_bias)
bias_subtracted_science = ccdproc.subtract_bias(science, master_bias)

In [ ]:
fig, axarr = plt.subplots(ncols=2, nrows=1, figsize=(12, 9))
images = [bias_subtracted_onebias,bias_subtracted_science]
cuts = [0,40,5000,8000]
for i in range(2):
    ax = axarr[i]
    vmin = cuts[0+2*i]
    vmax = cuts[1+2*i]
    img = ax.imshow(images[i].data, cmap='gray', origin='low',vmin=vmin,vmax=vmax)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.text(600, 100, images[i].header['FILENAME']+' - BIAS', fontsize=10, color='w')
    ax.text(200, 1800, 'using ccdproc.subtract_bias', fontsize=12, color='y')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(img, cax=cax) #, label='Number of counts')
    draw_rectangle(ax, images[i].data , 200, 500, 200, 500, color='w',text=True)

## Correcting all files from BIAS

In [ ]:
for i in range (len(filelist)):
    image = CCDData.read(filelist[i]) #, unit="adu")
    z_image = ccdproc.subtract_bias(image,master_bias)
    name_of_file = 'z'+ str(image.header['FILENAME'])
    z_image.header['FILENAME']  = name_of_file
    z_image.header['HISTORY']   = str(datetime.datetime.now())[0:18]+' astropy ccdproc substract_bias'
    z_image.header['HISTORY']   = 'using NOT2008/N1/zero_N1.fits master BIAS' 
    print('writting '+name_of_file+ ' in '+directory)
    z_image.write(directory+name_of_file,overwrite='yes')